In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import ipdb

In [ ]:
# mapping between abbreviated id, pedigree, male and female
workbook = 'HDP_MapFL18.xlsx'
sheet = 'ID_conversion'
mapping_df = pd.read_excel(workbook, sheet_name=sheet)
# list of all row indices which match the criteria
# idx = mapping_df.index[df['ID_abbreviated'] == '18R025'].tolist()
# mapping_df.iloc[idx[0]]['Male']

def get_parents(mapping_df, females_self_pollinating, females, hybrid):
    if hybrid == 'FILL' or hybrid == 'CHECK':
        return None, None
    
#     if hybrid == '83P17':
#         ipdb.set_trace()
    # there are some genotypes of only female
    if hybrid in females_self_pollinating:
        return None, females[females_self_pollinating.index(hybrid)]
    
    # return male and female of the parent hybrid plant
    idx = mapping_df.index[mapping_df['ID_abbreviated']==hybrid].tolist()
    idx1 = mapping_df.index[mapping_df['ID_abbreviated']=='PI'+hybrid].tolist()
    if len(idx) == 0 and len(idx1)==0:
        return None, None
    elem = idx[0] if len(idx1)==0 else idx1[0]
    male = mapping_df.iloc[elem]['Male']
    female = mapping_df.iloc[elem]['Female']
    if male == np.nan:
        male = None
    if female != female:
        female = None
    return male, female


In [3]:
# NOTE: not needed anymore
# the raw data files (originally csv files) are now stored in raw_data directory.

# data_dirs = '/home/sumit/old_stuffs/nifa/2018_field_season/ground_robot_phenotyping/30DAP/Phenotypes/'
# import os
# all_files = os.listdir(data_dirs)
# for f in all_files:
#     if f=='height_ground_robot_measures.csv':
#         dlim = ','
#     else:
#         dlim = '\t+'
#     df = pd.read_csv(data_dirs+f, delimiter=dlim, engine='python')
#     df.to_pickle('raw_data/'+f.split('.')[0] + '.pkl')

In [4]:
def get_mask(vec, ignores=[]):
    # False means the value is nan or one of the ignores
    mask = np.full(len(vec), True)
    for i, v in enumerate(vec):
        if v is np.nan or v in ignores:
            mask[i] = False
    return mask

data = mapping_df.values
columns = list(mapping_df)
male = data[:, columns.index('Male')]
female = data[:, columns.index('Female')]
hybrid = data[:, columns.index('ID_abbreviated')]
ignores = ['Check']
male_mask = get_mask(male, ignores)
female_mask = get_mask(female, ignores)
hybrid_mask = get_mask(hybrid, ignores)

# male mask and female mask are important, not sure about hybrid mask though
print(len(np.unique(female[female_mask])))
print(len(np.unique(male[male_mask])))
print(len(np.unique(hybrid[hybrid_mask])))

np.unique(female[female_mask])

# not sure if this genotype is male/female (it is in the females column though)
# but doesn't seem to have been crossed with any male
unknowns = ['83P17']
females_geno = ['ATx623', 'ATx642', 'ATxARG-1', 'PHA86']
females_self_pollinating = ['BTx623', 'BTx642', 'BTxARG-1', 'PHB86']

4
42
210


In [ ]:
data_file = 'raw_data/height_ground_robot_measures.pkl'
f_df = pd.read_pickle(data_file)

# make a dataframe with columns as Row1 Row2 Range ID_abbreviated Male Female
hybrids = f_df[['ID_abbreviated']].values.squeeze()
males = []
females = []
for h in hybrids:
    m, f = get_parents(mapping_df, females_self_pollinating + unknowns, females_geno + unknowns, h)
    males.append(m)
    females.append(f)
    
males = np.array(males)
females = np.array(females)

males_none = np.where(males==None)
males_none_hybrids = np.unique(hybrids[males_none])
# males None correponds to these hybrids -> ['83P17', 'BTx623', 'BTx642', 'BTxARG-1', 'FILL', 'PHB86']

all_males = np.unique(males[males!=None])
count_males = [np.sum(males==m) for m in all_males]
# a small test to determine if the hybrid to male detection is correct or not
for h, m in zip(hybrids, male):
    if m is None and h not in females_self_pollinating + unknowns + ['FILL']:
        print(h,m)

all_females = np.unique(females[females!=None])
count_females = [np.sum(females==f) for f in all_females]
print(all_females)
print(count_females)

> <ipython-input-2-7de32a4a7822>(16)get_parents()
     15     # there are some genotypes of only female
---> 16     if hybrid in females_self_pollinating:
     17         return None, females[females_self_pollinating.index(hybrid)]



In [14]:
df = pd.read_pickle('height_aerial_dataset.pkl')
# df.to_pickle('temp.pkl')
# df = pd.read_pickle('temp.pkl')


{'X': array([[ 2.,  1.,  1.,  0.,  0.,  0.],
        [ 4.,  1.,  1.,  0.,  0.,  0.],
        [ 6.,  1.,  1.,  0.,  0.,  0.],
        ...,
        [26., 37.,  0.,  1.,  0.,  0.],
        [28., 37.,  0.,  1.,  0.,  0.],
        [30., 37.,  0.,  0.,  0.,  1.]]), 'Y': array([[0.71940153],
        [0.66466346],
        [0.66116831],
        [0.67549336],
        [0.59337561],
        [0.53738017],
        [0.59462278],
        [0.74786076],
        [0.82677099],
        [0.79904545],
        [0.78062332],
        [0.65504562],
        [0.81423421],
        [0.81729332],
        [0.70123404],
        [0.73671861],
        [0.71916639],
        [0.70634813],
        [0.53480992],
        [0.54833242],
        [0.62140044],
        [0.67354334],
        [0.76649001],
        [0.94127571],
        [0.98948106],
        [0.91240797],
        [0.8443329 ],
        [0.75618824],
        [0.75836747],
        [0.70286316],
        [0.87629844],
        [0.75514369],
        [0.791495  ],
        [0

In [84]:
count_male = [(male==m).sum() for m in np.unique(male[male_mask])]
count_female = [(female==f).sum() for f in np.unique(female[female_mask])]
count_hybrid = [(hybrid==h).sum() for h in np.unique(hybrid[hybrid_mask])]

In [93]:
# group males (there are 42 of them) together using one phenotype


(42,)